# Challenge 2

In this challenge we will continue working with the `Pokemon` dataset. We will attempt solving a slightly more complex problem in which we will practice the iterative data analysis process you leaned in [this video](https://www.youtube.com/watch?v=xOomNicqbkk).

The problem statement is as follows:

**You are at a Pokemon black market planning to buy a Pokemon for battle. All Pokemon are sold at the same price and you can only afford to buy one. You cannot choose which specific Pokemon to buy. However, you can specify the type of the Pokemon - one type that exists in either `Type 1` or `Type 2`. Which type should you choose in order to maximize your chance of receiving a good Pokemon?**

To remind you about the 3 steps of iterative data analysis, they are:

1. Setting Expectations
1. Collecting Information
1. Reacting to Data / Revising Expectations

Following the iterative process, we'll guide you in completing the challenge.


## Problem Solving Iteration 1

In this iteration we'll analyze the problem and identify the breakthrough. The original question statement is kind of vague because we don't know what a *good pokemon* really means as represented in the data. We'll start by understanding the dataset and see if we can find some insights.

In [2]:
# Import libraries
import numpy as np
import pandas as pd

In [3]:
# Importing the dataset from Ironhack's database
# import libraries
import pandas as pd 
import sqlalchemy 
import datetime 
from sqlalchemy import create_engine
import numpy as np
import re

In [3]:
# import data set from Ironhack's database
driver = "mysql+pymysql"
user = "ironhacker_read"
password = "ir0nhack3r"
ip = "35.239.232.23"
database = "pokemon"

connection_string = f'{driver}://{user}:{password}@{ip}/{database}'

engine = create_engine(connection_string)
query = """
        SELECT * FROM pokemon_stats
"""

pokemon = pd.read_sql(query, engine)

KeyboardInterrupt: 

In [6]:
pokemon = pd.read_csv("pokemon.csv")

In [7]:
pokemon

,Unnamed: 0,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
5,5,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,1,False
6,6,6,Charizard,Fire,Flying,534,78,84,78,109,85,100,1,False
7,7,6,CharizardMega Charizard X,Fire,Dragon,634,78,130,111,130,85,100,1,False
8,8,6,CharizardMega Charizard Y,Fire,Flying,634,78,104,78,159,115,100,1,False
9,9,7,Squirtle,Water,NaN,314,44,48,65,50,64,43,1,False


From the data it seems whether a pokemon is good depends on its abilities as represented in the fields of `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total`. We are not sure about `Generation` and `Legendary` because they are not necessarily the decisive factors of the pokemon abilities.

But `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, `Speed`, and `Total` are a lot of fields! If we look at them all at once it's very complicated. This isn't Mission Impossible but it's ideal that we tackle this kind of problem after we learn Machine Learning (which you will do in Module 3). For now, is there a way to consolidate the fields we need to look into?

Fortunately there seems to be a way. It appears the `Total` field is computed based on the other 6 fields. But we need to prove our theory. If we can approve there is a formula to compute `Total` based on the other 6 abilities, we only need to look into `Total`.

We have the following expectation now:

#### The `Total` field is computed based on `HP`, `Attack`, `Defense`, `Sp. Atk`, `Sp. Def`, and `Speed`.

We need to collect the following information:

* **What is the formula to compute `Total`?**
* **Does the formula work for all pokemon?**

In the cell below, make a hypothesis on how `Total` is computed and test your hypothesis.

In [8]:
# your code here
pokemon["Total_test"]= pokemon["HP"] + pokemon["Attack"] + pokemon["Defense"] + pokemon["Sp. Atk"] + pokemon["Sp. Def"] + pokemon["Speed"]

In [9]:
pokemon["testing_sheme"] = pokemon["Total"] - pokemon["Total_test"] 

In [10]:
pokemon[pokemon["testing_sheme"]!=0]

,Unnamed: 0,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Total_test,testing_sheme


## Problem Solving Iteration 2

Now that we have consolidated the abilities fields, we can update the problem statement. The new problem statement is:

### Which pokemon type is most likely to have the highest `Total` value?

In the updated problem statement, we assume there is a certain relationship between the `Total` and the pokemon type. But we have two *type* fields (`Type 1` and `Type 2`) that have string values. In data analysis, string fields have to be transformed to numerical format in order to be analyzed. 

In addition, keep in mind that `Type 1` always has a value but `Type 2` is sometimes empty (having the `NaN` value). Also, the pokemon type we choose may be either in `Type 1` or `Type 2`.

Now our expectation is:

#### `Type 1` and `Type 2` string variables need to be converted to numerical variables in order to identify the relationship between `Total` and the pokemon type.

The information we need to collect is:

#### How to convert two string variables to numerical?

Let's address the first question first. You can use a method called **One Hot Encoding** which is frequently used in machine learning to encode categorical string variables to numerical. The idea is to gather all the possible string values in a categorical field and create a numerical field for each unique string value. Each of those numerical fields uses `1` and `0` to indicate whether the data record has the corresponding categorical value. A detailed explanation of One Hot Encoding can be found in [this article](https://hackernoon.com/what-is-one-hot-encoding-why-and-when-do-you-have-to-use-it-e3c6186d008f). You will formally learn it in Module 3.

For instance, if a pokemon has `Type 1` as `Poison` and `Type 2` as `Fire`, then its `Poison` and `Fire` fields are `1` whereas all other fields are `0`. If a pokemon has `Type 1` as `Water` and `Type 2` as `NaN`, then its `Water` field is `1` whereas all other fields are `0`.

#### In the next cell, use One Hot Encoding to encode `Type 1` and `Type 2`. Use the pokemon type values as the names of the numerical fields you create.

The new numerical variables you create should look like below:

![One Hot Encoding](../images/one-hot-encoding.png)

In [11]:
pokemon["Type 2"] = pokemon["Type 2"].fillna(value="")

In [12]:
# your code here

pokemon_type1 = pd.get_dummies(pokemon["Type 1"])
pokemon_type2 = pd.get_dummies(pokemon["Type 2"])
pokemon_type2 = pokemon_type2.drop(pokemon_type2.columns[0], axis=1)
pokemon_type_matrix = pokemon_type1 + pokemon_type2

# Second approach showed by Eva
# pokemon[["Type 1", "Type 2"]].stack().str.get_dummies().sum(level=0)
# .stack() --> turns into series, with two indexes, index1 is original one, index2 are the the types (type1 and type2, as rows)
# .str.get_dummies() --> creates like before the 1hot-encoding
# .sum(level=0) as we want to sum the rows of the level=1 index



In [13]:
pokemon_type_matrix

,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0
7,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


## Problem Solving Iteration 3

Now we have encoded the pokemon types, we will identify the relationship between `Total` and the encoded fields. Our expectation is:

#### There are relationships between `Total` and the encoded pokemon type variables and we need to identify the correlations.

The information we need to collect is:

#### How to identify the relationship between `Total` and the encoded pokemon type fields?

There are multiple ways to answer this question. The easiest way is to use correlation. In the cell below, calculate the correlation of `Total` to each of the encoded fields. Rank the correlations and identify the #1 pokemon type that is most likely to have the highest `Total`.

In [14]:
# your code here
new_matrix = pokemon_type_matrix
new_matrix["Total"] = pokemon["Total"]

In [15]:
test = new_matrix.corr(method="pearson")

In [16]:
test = (test[["Total"]]).drop("Total")

In [17]:
test

,Total
Bug,-0.145781
Dark,0.056154
Dragon,0.229705
Electric,0.020971
Fairy,-0.036698
Fighting,0.077786
Fire,0.078726
Flying,0.059383
Ghost,0.003641
Grass,-0.052592


In [18]:
Correlations = test.rename(columns = {"Total": "Corr"})

In [19]:
Correlations["Corr"].sort_values(ascending=False)

Dragon      0.229705
Psychic     0.124688
Steel       0.109703
Fire        0.078726
Fighting    0.077786
Ice         0.060248
Flying      0.059383
Dark        0.056154
Rock        0.032731
Electric    0.020971
Ground      0.015060
Ghost       0.003641
Water      -0.021665
Fairy      -0.036698
Grass      -0.052592
Poison     -0.090441
Normal     -0.105331
Bug        -0.145781
Name: Corr, dtype: float64

# Bonus Question

Say now you can choose both `Type 1` and `Type 2` of the pokemon. In order to receive the best pokemon, which types will you choose?

In [45]:
# your code here
pokemon["Total Type"] = (pokemon["Type 1"] + "-" + pokemon["Type 2"]).apply(lambda x : re.sub("-$", "", x))

In [46]:
pokemon_TypeVsTotal =pokemon[["Name", "Total Type", "Total"]]

In [49]:
pokemon_TypeVsTotal_Encr = pd.get_dummies(pokemon_TypeVsTotal["Total Type"])

In [51]:
pokemon_TypeVsTotal_Encr["Total"] = pokemon_TypeVsTotal["Total"]

In [54]:
pokemon_Type_VsTotal_Encr = ((pokemon_TypeVsTotal_Encr.corr(method="pearson"))["Total"]).drop("Total")

In [62]:
pokemon_Type_VsTotal_Encr.sort_values(ascending=False)

Dragon-Flying       0.149777
Dragon-Ice          0.128737
Dragon-Psychic      0.127066
Psychic-Fighting    0.104173
Ghost-Dragon        0.102264
Ground-Fire         0.098824
Rock-Dark           0.089736
Fire-Flying         0.084398
Rock-Fairy          0.084386
Water-Dragon        0.073033
Psychic-Dark        0.072266
Dragon-Fire         0.072266
Dragon-Electric     0.072266
Steel-Dragon        0.072266
Electric-Flying     0.067801
Normal-Fighting     0.064682
Water-Fighting      0.062210
Flying              0.060506
Ground-Flying       0.059068
Fire-Dragon         0.058692
Bug-Steel           0.058471
Fighting-Steel      0.058418
Grass-Dragon        0.057512
Dragon-Ground       0.057482
Psychic             0.054856
Rock-Flying         0.052859
Steel-Ground        0.052154
Electric-Dragon     0.051610
Fire-Water          0.048659
Psychic-Ghost       0.048659
                      ...   
Grass-Fairy        -0.023010
Ice-Psychic        -0.023010
Ghost-Grass        -0.023651
Ghost-Fire    